In [70]:
# Having a look at the SAMI stellar kinematic data and possibly reducing it!

import astropy.io.fits as fits
import numpy as np
from astropy.table import unique,Table,setdiff
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt

# Convert FITS data into astropy table
samiDR3Stelkin = Table.read('/suphys/fscu5535/python_scripts/raw_data/samiDR3Stelkin.fits', format='fits',memmap=True)
GAMA_galaxies_in_SAMI = Table.read('/suphys/fscu5535/python_scripts/mod_data/GAMA_galaxies_in_SAMI.fits', format='fits',memmap=True)
cube_list_ISBEST = Table.read('cube_list_ISBEST.fits',format='fits',memmap=True)


# Display basic table info
print("Table info:","\n")
samiDR3Stelkin.info()
print("\n\n")
cube_list_ISBEST.info()
print("\n\n")



Table info: 

<Table length=3426>
          name           dtype  n_bad
----------------------- ------- -----
                 CUBEID bytes70     0
              CUBEIDPUB bytes12     0
               CUBENAME bytes71     0
                  CATID   int64     0
             PA_STELKIN float32   611
         PA_STELKIN_ERR float32   611
               SIGMA_RE float32  1175
           SIGMA_RE_ERR float32  1175
       SIGMA_3KPC_ROUND float32   799
   SIGMA_3KPC_ROUND_ERR float32   799
    SIGMA_1_4_ARCSECOND float32  1019
SIGMA_1_4_ARCSECOND_ERR float32  1019
      SIGMA_2_ARCSECOND float32   879
  SIGMA_2_ARCSECOND_ERR float32   879
      SIGMA_3_ARCSECOND float32   801
  SIGMA_3_ARCSECOND_ERR float32   801
      SIGMA_4_ARCSECOND float32   775
  SIGMA_4_ARCSECOND_ERR float32   775
         APER_CORR_FLAG float32  1468
         RMAX_APER_CORR float32  1468
             LAMBDAR_RE float32  1496
         LAMBDAR_RE_ERR float32  1496
            LAMBDAR_MAX float32  2989
        LAMBDAR_

In [17]:
# Find all the rows of CATAID's that aren't in both GAMA and SAMI. Count is there as a check
rows_to_remove = list()
count = 0
for i in range(len(cube_list_ISBEST)):
    inBoth = False
    for j in range(len(GAMA_galaxies_in_SAMI)):
        if (cube_list_ISBEST['CATID'][i] == GAMA_galaxies_in_SAMI['CATAID'][j]) and inBoth == False :
            inBoth = True
            count +=1
    if inBoth == False:
        rows_to_remove.append(i)
print(count,"matching galaxies found!")
print(len(rows_to_remove),"to be removed")

# Now remove the rows from cube_list_ISBEST
rows_to_remove_table = Table()
rows_to_remove_table['row_number'] = rows_to_remove
cube_list_ISBEST.remove_rows(rows_to_remove_table['row_number'])
print(len(cube_list_ISBEST),"galaxies retained.")

# Sort by CATID and write the reduced table to file
cube_list_ISBEST.sort('CATID')
cube_list_ISBEST.write('/suphys/fscu5535/python_scripts/mod_data/cube_list_ISBEST_in_GAMA.fits',format='fits')



2147 matching galaxies found!
1098 to be removed


In [50]:
# Now compare cube_list_ISBEST_in_GAMA with GAMA_galaxies_in_GAMA to make sure everything is correct
cube_list_ISBEST_in_GAMA = Table.read('/suphys/fscu5535/python_scripts/mod_data/cube_list_ISBEST_in_GAMA.fits',format='fits',memmap=True)
matching = 0
for i in range(len(GAMA_galaxies_in_SAMI)):
    if GAMA_galaxies_in_SAMI['CATAID'][i] == cube_list_ISBEST_in_GAMA['CATID'][i]:
        matching += 1
print(matching)

2147


In [167]:
# Now we want to remove all the rows in samiDR3Stelkin that don't have a matching CUBEID in cube_list_ISBEST_in_GAMA
samiDR3Stelkin = Table.read('/suphys/fscu5535/python_scripts/raw_data/samiDR3Stelkin.fits',format='fits',memmap=True)
import string
rows_to_remove = list()
for i in range(len(samiDR3Stelkin)):
    matching = False
    for j in range(len(cube_list_ISBEST_in_GAMA)):
        if matching == False and cube_list_ISBEST_in_GAMA['CUBENAME'][j] == samiDR3Stelkin['CUBENAME'][i].strip():
            matching = True
    if matching == False:
        rows_to_remove.append(i)
print("Matching complete,",len(rows_to_remove),"to be removed")
print("Number of rows that will remain:",len(samiDR3Stelkin)-len(rows_to_remove))

# Now remove the rows from samiDR3Stelkin and write to file
rows_to_remove_table = Table()
rows_to_remove_table['row_number'] = rows_to_remove
samiDR3Stelkin.remove_rows(rows_to_remove_table['row_number'])
print(len(samiDR3Stelkin),"galaxies retained.")
samiDR3Stelkin.write('/suphys/fscu5535/python_scripts/mod_data/samiDR3Stelkin_in_GAMA.fits',format='fits',overwrite=True)
        

Matching complete, 1279 to be removed
Number of rows that will remain: 2147


In [173]:
# Now compare samiDR3Stelkin_in_GAMA with GAMA_galaxies_in_GAMA to make sure everything is correct
samiDR3Stelkin_in_GAMA = Table.read('/suphys/fscu5535/python_scripts/mod_data/samiDR3Stelkin_in_GAMA.fits',format='fits',memmap=True)
samiDR3Stelkin_in_GAMA.sort('CATID')
cube_list_ISBEST_in_GAMA.sort('CATID')
matching_stelkin_cube = 0
matching_stelkin_g3cgal = 0
for i in range(len(samiDR3Stelkin_in_GAMA)):
    if samiDR3Stelkin_in_GAMA['CATID'][i] == cube_list_ISBEST_in_GAMA['CATID'][i]:
        matching_stelkin_cube += 1
    if samiDR3Stelkin_in_GAMA['CATID'][i] == GAMA_galaxies_in_SAMI['CATAID'][i]:
        matching_stelkin_g3cgal += 1
print("stelkin and cube_list:",matching_stelkin_cube)
print("stelkin and g3cgal:",matching_stelkin_g3cgal)

stelkin and cube_list: 2147
stelkin and g3cgal: 2147
